# Исследование надежности заемщиков


## Изучение общей информации о данных

In [2]:
import pandas as pd

try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
#Функция для перевода долей в проценты
def percents(column):
    return str(round((column.sum() / column.count()) * 100, 3)) + '%'

#Сводная таблица по стобцам 'children' и 'debt'
pd.pivot_table(data,
               index=['children'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1063,14091,7.544%
3,27,330,8.182%
1,444,4808,9.235%
2,194,2052,9.454%
4,4,41,9.756%


**Вывод:** Наименьшая вероятность просрочки платежа у клиентов без детей, наибольшая - у клиентов с 4 детьми. Клиенты с 5 детьми вообще не имеют просрочек, но будет не корректно оценивать данные на основе 9 записей, поэтому нужно больше информации по таким семьям. Основная зкономерность, что чем больше детей, тем выше вероятность просрочки платежа по кредиту.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
#Сводная таблица по столбцам 'family_status' и 'debt'
pd.pivot_table(data,
               index=['family_status'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
family_status,,,
вдовец / вдова,63,951,6.625%
в разводе,84,1189,7.065%
женат / замужем,927,12261,7.561%
гражданский брак,385,4134,9.313%
Не женат / не замужем,273,2796,9.764%


**Вывод:** Наиболее ответственные заемщики из категорий "вдовец/вдова", "в разводе", "женат/замужем". Вероятность возникновения просрочки платежа составляет в среднем 7%. Наименее ответственные из категории "не женат/не замужем". Они не платят в срок с вероятностью около 10%.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# Сводная таблица по столбцам 'total_income_category', и 'debt'
pd.pivot_table(data,
               index=['total_income_category'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
total_income_category,,,
D,21,349,6.017%
B,354,5014,7.06%
A,2,25,8.0%
C,1353,15921,8.498%
E,2,22,9.091%


**Вывод:** Люди из категорий А, Е и D берут меньше всего кредитов, поэтому исключу из рассмотрения в связи с недостатком данных. Менее надежные клиенты из категории С с вероятностью - 8.5%, но в тоже время составляют подавляющее большинство заемщиков. Наиболее надежные клиенты из категории B.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [26]:
# Сводная таблица по столбцам 'purpose_category' и 'debt'
pd.pivot_table(data,
               index=['purpose_category'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,780,10751,7.255%
проведение свадьбы,183,2313,7.912%
получение образования,369,3988,9.253%
операции с автомобилем,400,4279,9.348%


**Вывод:** Меньше всего пропускают платежи клиенты с ипотечной задолженностью, всего в 7% случаях. Интересно, что операции с недвижимостью занимают наибольший сегмент кредитов. Клиенты с кредитом на автомобиль и образование напротив, чаще всего пропускают платежи, в 9% случаях.

### Шаг 4: общий вывод.

На основе полученных данных можно обрисовать ответственного заемщика: клиент берущий ипотечный кредит, без детей, ранне состоял или состоит в браке, со средним заработком.

Рассмотрю данные подробнее.

1. Зависимость от количества детей.

In [26]:
#Функция для перевода долей в проценты
def percents(column):
    return str(round((column.sum() / column.count()) * 100, 3)) + '%'

#Сводная таблица по стобцам 'children' и 'debt'
pd.pivot_table(data,
               index=['children'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1063,14091,7.544%
3,27,330,8.182%
1,444,4808,9.235%
2,194,2052,9.454%
4,4,41,9.756%


На таблице выше видно, что наиболее лучшие заемщики без детей. Самые ненадежные - у кого 2 детей. Группы людей с 3, 4 и 5 детьми не рассматриваются из-за нехватки данных. Наблюдается закономерность: меньше детей - меньше просрочек.

2. Зависимость от семейного положения.

In [27]:
#Сводная таблица по столбцам 'family_status' и 'debt'
pd.pivot_table(data,
               index=['family_status'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
family_status,,,
вдовец / вдова,63,951,6.625%
в разводе,84,1189,7.065%
женат / замужем,927,12261,7.561%
гражданский брак,385,4134,9.313%
Не женат / не замужем,273,2796,9.764%


На основе этих данных самые надежные состоящие или состоявшие в браке, порядка 7% просрочек. При чем колличество женатых или замужних заемщиков в разы превосходит остальные категории. Могу предположить, что большая часть задолженности будет ипотечной. Наименее привлекательны для банка будут не женатые и не замужние граждане, порядка 10 %.

3. Зависимость от уровня дохода.

In [28]:
# Сводная таблица по столбцам 'total_income_category', и 'debt'
pd.pivot_table(data,
               index=['total_income_category'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
total_income_category,,,
D,21,349,6.017%
B,354,5014,7.06%
A,2,25,8.0%
C,1353,15921,8.498%
E,2,22,9.091%


На основании вышеизложенных данных смело исключаю категории A, D и Е, так как для этих категорий недостаточно данных. Судя по всему, для людей с очень высоким и очень низким доходами мало интересен кредит, так как одним он вообще не нужен, а другие относятся с осторожностью. Основной контингент - люди из категорий В и С. Отличия по возможным просрочкам всего 1.5%. Но все же клиенты из категории В более привлекательны чем из С.

4. Зависимость от цели кредита.

In [29]:
# Сводная таблица по столбцам 'purpose_category' и 'debt'
pd.pivot_table(data,
               index=['purpose_category'],
               values=['debt'],
               aggfunc=['sum', 'count', percents]).sort_values(by=('percents', 'debt'))

,sum,count,percents
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,780,10751,7.255%
проведение свадьбы,183,2313,7.912%
получение образования,369,3988,9.253%
операции с автомобилем,400,4279,9.348%


Охотнее всего платят в срок заемщики с ипотечным кредитом. Просрочки всего в 7% случаев, несмотря на то, что это самый популярный кредит. Менее охотно платят люди с кредитом на образование или автомобиль. Просрочки уже, примерно, в 9% случаев.